In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
tf.set_random_seed(777)

# < Data Loading >

#Pandas 이용하여 DATA 불러오기 / DATA 살펴보기

data = pd.read_csv('AMZN.csv').set_index('Date').astype(np.float)
data.index = pd.to_datetime(data.index,format = '%Y-%m-%d')
print(data.info())   

# DataFrame을 tensorflow 연산을 위해 Array로 변경 

data = data.values[0:]
print("data shape: ", data.shape)

# < DATA 전처리 >
## 정규화 방식 ##

# 1. Standardization
def data_standardization(x):
    x_np = np.asarray(x)
    return (x_np - x_np.mean()) / x_np.std()

# 2. Min-Max scaling
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np - x_np.min()) / (x_np.max() - x_np.min() + 1e-7) # 1e-7은 0으로 나누는 오류 예방차원

# 3. Reverse_min_max_scaling
def reverse_min_max_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()


## Input으로 사용되는 가격과 거래량 데이터의 스케일 차이가 크기때문에 각각 정규화 ##


# 가격데이터 정규화
# ['Open','High','Low','Volume','Close']에서 'Low' 까지
price = data[:,:-2]
norm_price = min_max_scaling(price) 
print("price.shape: ", price.shape)
print("price[0]: ", price[0])
print("norm_price[0]: ", norm_price[0])
print("-"*100)

# 거래량형태 데이터를 정규화한다
# ['Open','High','Low','Volume','Close']에서 'Volume'
volume = data[:,-2:-1]
norm_volume =data_standardization(volume) 
print("volume.shape: ", volume.shape)
print("volume[0]: ", volume[0])
print("norm_volume[0]: ", norm_volume[0])
print("-"*100)

# price, volume data 재결합
x = np.concatenate([norm_price, norm_volume], axis=1) 
print("x.shape: ", x.shape)
print("x[0]: ", x[0])    # x의 첫 값
print("x[-1]: ", x[-1])  # x의 마지막 값
print("-"*100)

# y(정답set)는 'Close'
close = data[:,-1:]
y = min_max_scaling(close)
print("y[0]: ",y[0])     # y의 첫 값
print("y[-1]: ",y[-1])   # y의 마지막 값

# 하이퍼파라미터 설정

input_data_column_num = 4  # 입력데이터의 컬럼 개수(Variable 개수) = Input dimension
output_data_column_num = 1 # 결과데이터의 컬럼 개수 

seq_length = 28          # 1개 시퀀스의 길이(시계열데이터 입력 개수)
rnn_cell_hidden_dim = 20   # 각 셀의 (hidden)출력 크기
forget_bias = 1.0          # 망각편향(기본값 1.0)
num_stacked_layers = 1    # stacked LSTM layers 개수
keep_prob = 1             # dropout할 때 keep할 비율

epoch_num = 1000           # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01       # 학습률

# 입출력용 DATA setting 

dataX = [] # 입력으로 사용될 Sequence Data
dataY = [] # 출력(타켓)으로 사용

for i in range(0, len(y) - seq_length):
    _x = x[i : i+seq_length]
    _y = y[i + seq_length] # 다음 나타날 주가(정답)
    
    dataX.append(_x) # dataX 리스트에 추가
    dataY.append(_y) # dataY 리스트에 추가

# 학습용/테스트용 데이터 생성

# 전체 train:test 70% : 30%
train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size

# 학습용 데이터 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])

# 테스트용 데이터 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])

# 텐서플로우 플레이스홀더 생성
X = tf.placeholder(tf.float32, [None, seq_length, input_data_column_num])
print("X: ", X)
Y = tf.placeholder(tf.float32, [None, 1])
print("Y: ", Y)

# 검증용 측정지표를 산출하기 위한 targets, predictions를 생성한다
targets = tf.placeholder(tf.float32, [None, 1])
print("targets: ", targets)

predictions = tf.placeholder(tf.float32, [None, 1])
print("predictions: ", predictions)

# 모델(LSTM RNN) 생성 + FC layers

def LSTM_cell():
    
    # forget_bias: biases of the forget gate (default: 1) in order to reduce the scale of forgetting in the beginning of the training.
    
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim, 
                                         state_is_tuple=True, activation=None)
#     if keep_prob < 1.0:
#         cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell

# num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
stackedRNNs = [LSTM_cell() for _ in range(num_stacked_layers)]
multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) 

# RNN Cell(여기서는 LSTM셀임)들을 연결
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("outputs: ", outputs)

# [:, -1]를 잘 살펴보자. LSTM RNN의 마지막 (hidden)출력만을 사용했다.
# 과거 여러 거래일의 주가를 이용해서 다음날의 주가 1개를 예측하기때문에 MANY-TO-ONE형태이다
Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_data_column_num, activation_fn=None)

# Loss function define
loss = tf.reduce_sum(tf.square(Y_pred - Y))

# Optimizer
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# RMSE(Root Mean Square Error)
rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))

# Learning 

train_error_summary = [] # 학습용 데이터의 오류를 중간 중간 기록한다
test_error_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다
test_predict = ''        # 테스트용데이터로 예측한 결과

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print('Learning Start!')
    
    for epoch in range(epoch_num):
        _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
        
        if ((epoch+1) % 200 == 0) or (epoch == epoch_num-1): # 100번째마다 또는 마지막 epoch인 경우
            
            # 학습용데이터로 rmse오차를 구한다
            train_predict = sess.run(Y_pred, feed_dict={X: trainX})
            train_error = sess.run(rmse, feed_dict={targets: trainY, predictions: train_predict})
            train_error_summary.append(train_error)

            # 테스트용데이터로 rmse오차를 구한다
            test_predict = sess.run(Y_pred, feed_dict={X: testX})
            test_error = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
            test_error_summary.append(test_error)

            # 현재 오류를 출력한다
            print("epoch: {}, train_error(A): {}, test_error(B): {}, B-A: {}".format(epoch+1, train_error, test_error, test_error-train_error))
    
    recent_data = np.array([x[len(x)-seq_length : ]])  
    
    # 내일 종가를 예측해본다
    ostest_predict = sess.run(Y_pred, feed_dict={X: recent_data})

    print("ostest_predict", ostest_predict[0])
    ostest_predict = reverse_min_max_scaling(close,ostest_predict) # 금액데이터 역정규화한다
    print("Predicted stock price", ostest_predict[0]) # 예측한 주가를 출력한다
    print("Real price : " , close[len(close)-seq_length : ][0])
    
# 결과 그래프 출력
plt.figure(1)
plt.plot(train_error_summary, 'gold',label='train error')
plt.plot(test_error_summary, 'b',label='test error')
plt.title('Cost')
plt.xlabel('Epoch(x100)')
plt.ylabel('Root Mean Square Error')
plt.legend(loc = 'best')

plt.figure(2)
plt.plot(testY, 'r',label='Real price in test set')
plt.plot(test_predict, 'b',label='Prediction price')
plt.title('Real vs Test set predict')
plt.xlabel('Time Period')
plt.ylabel('Stock Price')
plt.legend(loc='best')
plt.show()    